In [1]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

import matplotlib.pyplot as plt

# import xgboost as xgb

In [2]:
# !pip install xgboost

In [3]:
# !pip install pycaret==2.3.10 markupsafe==2.0.1 pyyaml==5.4.1 -qq 


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


seed_everything(42)  # Seed 고정

In [5]:
class CFG:
    dataPath = "./open/"
    trainPath = dataPath+'train.csv'
    testPath = dataPath+'test.csv'
    submission = dataPath+'sample_submission.csv'
    outPath = './output/'
    weightsavePath = './weights/'

In [11]:
train_df = pd.read_csv(CFG.trainPath)
train_x = train_df.filter(regex='X')  # Input : X Featrue
train_y = train_df.filter(regex='Y')  # Output : Y Feature
test_df = pd.read_csv(CFG.testPath)

In [17]:
test_df_drop = test_df.drop('ID',axis=1)
test_df_drop

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,0.0,4.37,0.15,13.38,13.40,13.50,13.53,13.46,3.27,3.32,3.18,3.39,1,2.22,2.14,2.31,2.16,2.32,2.22,1.38,1.53,1.33,1.53,12.96,12.95,12.97,12.88,-16.17,-16.17,-16.84,21.22,21.06,21.21,21.20,0.16,1469,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
1,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,0.0,4.36,0.17,13.35,13.38,13.45,13.52,13.44,3.26,3.30,3.17,3.34,1,2.11,2.10,2.10,2.10,2.17,2.20,1.36,1.56,1.34,1.55,12.94,12.89,13.00,12.88,-15.35,-15.25,-16.01,21.21,20.99,21.26,21.15,0.27,1462,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
2,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,0.0,4.43,0.20,13.32,13.35,13.40,13.52,13.41,3.07,3.12,3.05,3.17,1,2.13,2.10,2.29,2.15,2.24,2.26,1.50,1.69,1.47,1.71,12.93,12.90,13.02,13.01,-16.27,-16.35,-16.95,21.16,21.05,21.15,21.19,0.14,1469,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
3,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,0.0,4.39,0.18,13.32,13.34,13.45,13.50,13.42,3.23,3.11,3.20,3.14,1,2.10,2.10,2.09,2.09,2.10,2.18,1.49,1.65,1.46,1.90,12.99,12.98,12.91,12.96,-16.06,-16.13,-16.75,21.15,21.03,21.16,21.14,0.13,1469,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
4,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,0.0,4.38,0.12,13.38,13.39,13.45,13.50,13.46,3.17,3.20,3.04,3.20,1,2.12,2.11,2.10,2.10,2.12,2.18,1.48,1.73,1.46,1.67,13.01,12.84,12.99,13.01,-16.06,-16.17,-16.76,21.15,21.08,21.12,21.17,0.09,1469,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,0.0,0.0,4.35,0.14,13.38,13.37,13.48,13.51,13.45,3.16,3.06,3.07,3.09,1,2.13,2.08,2.09,2.07,2.12,2.16,1.37,1.66,1.36,1.56,12.98,13.00,12.91,12.90,-16.17,-16.26,-16.88,21.16,21.13,21.24,21.19,0.11,1469,1,1,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843
39604,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,0.0,0.0,4.34,0.16,13.37,13.38,13.47,13.53,13.46,3.18,2.98,3.09,3.06,1,2.06,2.05,2.05,2.08,2.08,2.14,1.36,1.64,1.36,1.68,12.92,12.95,12.99,13.00,-16.11,-16.10,-16.73,21.16,21.03,21.22,21.12,0.19,1469,1,1,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614
39605,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,0.0,0.0,4.38,0.16,13.36,13.36,13.46,13.52,13.44,3.18,3.02,3.09,3.07,1,2.11,2.13,2.06,2.11,2.12,2.18,1.40,1.62,1.35,1.72,12.99,12.88,13.01,12.85,-16.23,-16.32,-16.93,21.16,21.11,21.23,21.17,0.12,1469,1,1,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108
39606,67.485,103.321,61.77,1,103.148,67.845,32.05,115.05,267.26,0.0,0.0,4.37,0.16,13.38,13.37,13.49,13.53,13.46,3.14,3.06,3.02,3.11,1,2.11,2.12,2.09,2.12,2.12,2.20,1.38,1.56,1.37,1.59,12.97,13.00,12.99,12.90,-15.99,-16.05,-16.67,21.18,21.10,21.21,21.19,0.11,1469,1,1,60236.73,121.110646,125.471699,134.989984,120.889578,129.296909,132.673977,131.882893


In [7]:
from pycaret.regression import *
SEED = 42
# train_df
# Y = ['X_01','X_02']
train_df

,ID,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TRAIN_00001,70.544,103.320,67.47,1,101.892,74.983,29.45,62.38,245.71,0.0,0.0,4.34,0.18,13.34,13.37,13.40,13.52,13.44,3.11,3.17,3.06,3.13,1,2.05,2.07,2.05,2.06,2.06,2.35,1.49,1.69,1.46,1.74,12.99,12.88,12.89,12.99,-16.41,-16.36,-17.03,21.20,20.99,21.28,21.09,0.29,1463,1,1,9706.03,137.043591,135.359219,147.837968,134.313475,125.605427,136.721425,125.028256,2.056,1.456,1.680,10.502,29.632,16.083,4.276,-25.381,-25.529,-22.769,23.792,-25.470,-25.409,-25.304
1,TRAIN_00002,69.524,103.321,65.17,1,101.944,72.943,28.73,61.23,233.61,0.0,0.0,4.38,0.18,13.33,13.33,13.46,13.51,13.42,2.97,3.11,2.91,3.20,1,2.10,2.10,2.10,2.10,2.17,2.28,1.49,1.67,1.45,1.66,12.92,12.87,12.89,12.93,-16.06,-16.11,-16.74,21.16,21.03,21.16,21.13,0.13,1463,1,1,10423.43,133.736691,135.979817,149.924692,123.630583,127.893337,143.322659,124.877308,1.446,1.184,1.268,18.507,33.179,16.736,3.229,-26.619,-26.523,-22.574,24.691,-26.253,-26.497,-26.438
2,TRAIN_00003,72.583,103.320,64.07,1,103.153,72.943,28.81,105.77,272.20,0.0,0.0,4.36,0.15,13.36,13.36,13.39,13.51,13.43,3.04,3.04,3.01,3.12,1,2.07,2.06,2.04,2.07,2.10,2.26,1.49,1.69,1.46,1.68,12.97,12.87,12.87,13.00,-16.16,-16.17,-16.76,21.13,21.03,21.17,21.12,0.14,1468,1,1,10948.53,132.805112,131.055355,146.814592,128.939070,127.012195,140.395688,122.238232,1.251,0.665,0.782,14.082,31.801,17.080,2.839,-26.238,-26.216,-22.169,24.649,-26.285,-26.215,-26.370
3,TRAIN_00004,71.563,103.320,67.57,1,101.971,77.022,28.92,115.21,255.36,0.0,0.0,4.33,0.21,13.30,13.33,13.39,13.51,13.40,3.05,3.01,3.02,3.08,1,2.06,2.05,2.03,2.06,2.07,2.12,1.47,1.68,1.47,1.68,12.91,12.97,12.99,12.92,-16.05,-16.03,-16.67,21.18,20.98,21.20,21.09,0.22,1469,1,1,15007.03,134.138760,133.239422,139.720132,132.260824,130.723186,147.624829,134.875225,1.464,1.079,1.052,16.975,34.503,17.143,3.144,-25.426,-25.079,-21.765,24.913,-25.254,-25.021,-25.345
4,TRAIN_00005,69.524,103.320,63.57,1,101.981,70.904,29.68,103.38,241.46,0.0,0.0,4.35,0.16,13.35,13.34,13.41,13.50,13.42,3.04,3.07,3.00,3.12,1,2.09,2.06,2.09,2.12,2.33,2.13,1.49,1.68,1.47,1.82,12.96,12.85,12.91,12.96,-16.25,-16.23,-16.85,21.16,20.96,21.18,21.10,0.22,1469,1,1,11051.03,142.728970,136.620022,134.853555,134.760252,125.647793,139.331105,123.272762,0.983,0.646,0.689,15.047,32.602,17.569,3.138,-25.376,-25.242,-21.072,25.299,-25.072,-25.195,-24.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,TRAIN_39603,66.465,103.320,62.27,1,103.150,66.825,30.20,77.83,298.05,0.0,0.0,4.36,0.15,13.37,13.38,13.51,13.52,13.46,3.20,3.03,3.06,3.13,1,2.10,2.11,2.08,2.11,2.10,2.20,1.37,1.60,1.36,1.67,12.89,13.00,12.94,12.90,-16.09,-16.15,-16.79,21.17,21.08,21.19,21.19,0.11,1469,1,1,60630.73,129.965741,130.807148,133.481737,125.273130,121.780933,133.780110,129.029812,1.382,1.215,1.263,10.874,29.194,16.582,3.410,-26.486,-26.581,-22.772,24.261,-26.491,-26.584,-26.580
39603,TRAIN_39604,66.465,103.321,62.77,1,102.021,66.825,29.21,102.25,270.67,0.0,0.0,4.40,0.13,13.36,13.36,13.47,13.49,13.44,3.15,3.06,3.05,3.06,1,2.12,2.12,2.09,2.12,2.11,2.20,1.40,1.68,1.37,1.77,12.88,12.88,12.89,12.86,-15.70,-15.74,-16.40,21.16,21.09,21.21,21.19,0.12,1458,1,1,60763.43,127.633885,120.158764,142.667802,122.465490,122.987209,143.090741,122.811413,1.482,0.606,1.083,8.759,29.859,15.659,3.406,-27.308,-27.203,-24.674,23.427,-27.250,-27.334,-27.325
39604,TRAIN_39605,68.504,103.320,64.67,1,103.144,68.864,29.96,102.61,198.07,0.0,0.0,4.38,0.14,13.38,13.39,13.47,13.52,13.46,3.23

In [8]:
for i in range(1,10):
    globals()['train_{}'.format(i)] = pd.concat([train_x, train_y['Y_0'+str(i)]], axis=1)

for i in range(10,15):
    globals()['train_{}'.format(i)] = pd.concat([train_x, train_y['Y_'+str(i)]], axis=1)

# train_1 = pd.concat([train_x, train_y['Y_01']], axis=1)
# train_2 = pd.concat([train_x, train_y['Y_02']], axis=1)
# train_3 = pd.concat([train_x, train_y['Y_03']], axis=1)
# train_4 = pd.concat([train_x, train_y['Y_04']], axis=1)
# train_5 = pd.concat([train_x, train_y['Y_05']], axis=1)
# train_6 = pd.concat([train_x, train_y['Y_06']], axis=1)
# train_7 = pd.concat([train_x, train_y['Y_07']], axis=1)
# train_8 = pd.concat([train_x, train_y['Y_08']], axis=1)
# train_9 = pd.concat([train_x, train_y['Y_09']], axis=1)
# train_10 = pd.concat([train_x, train_y['Y_10']], axis=1)
# train_11 = pd.concat([train_x, train_y['Y_11']], axis=1)
# train_12 = pd.concat([train_x, train_y['Y_12']], axis=1)
# train_13 = pd.concat([train_x, train_y['Y_13']], axis=1)
# train_14 = pd.concat([train_x, train_y['Y_14']], axis=1)

In [10]:
for idx in range(1,10):
    setup(globals()['train_{}'.format(idx)], target = 'Y_0'+str(idx), session_id=SEED ,silent=True)
    best = compare_models(sort='RMSE',fold=5, n_select=2)
    tuned_best = [tune_model(i, optimize='RMSE',n_iter = 3) for i in best]
    blend_best = blend_models(estimator_list=tuned_best,optimize='RMSE')
    globals()['final_model_{}'.format(idx)] = finalize_model(blend_best)
    print(f'complete :{idx}')
    # f = finalize_model(best_model)
    
#     # attach final model to a dictionary
#     final_model[i] = f
    save_model(globals()['final_model_{}'.format(idx)],model_name='model'+str(idx),verbose=False)
#     # save transformation pipeline and model as pickle file 
#     save_model(f, model_name='trained_models/' + str(i), verbose=False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4978,0.4128,0.6425,0.0617,0.0234,0.0189
1,0.4880,0.4019,0.6339,0.0672,0.0231,0.0185
2,0.4910,0.3990,0.6317,0.0572,0.0230,0.0186
3,0.4909,0.3941,0.6278,0.0590,0.0229,0.0186
4,0.5001,0.4068,0.6378,0.0475,0.0232,0.0189
5,0.4769,0.3803,0.6167,0.0587,0.0225,0.0181
6,0.4919,0.3955,0.6289,0.0570,0.0229,0.0187
7,0.4921,0.4023,0.6342,0.0591,0.0231,0.0186
8,0.5014,0.4110,0.6411,0.0567,0.0234,0.0190


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 

In [12]:
# for idx in range(10,15):
#     setup(globals()['train_{}'.format(idx)], target = 'Y_'+str(idx), session_id=SEED )
#     best = compare_models(sort='RMSE', n_select=2)
#     tuned_best = [tune_model(i, optimize='RMSE') for i in best]
#     blend_best = blend_models(estimator_list=tuned_best,optimize='RMSE')
#     globals()['final_model_{}'.format(idx)] = finalize_model(blend_best)
#     # f = finalize_model(best_model)
    
# #     # attach final model to a dictionary
# #     final_model[i] = f
#     save_model(globals()['final_model_{}'.format(idx)],model_name='model'+str(idx),verbose=False)
# #     # save transformation pipeline and model as pickle file 
# #     save_model(f, model_name='trained_models/' + str(i), verbose=False)

In [13]:
for idx in range(10,15):
    setup(globals()['train_{}'.format(idx)], target = 'Y_'+str(idx), session_id=SEED ,silent=True)
    best = compare_models(sort='RMSE',fold=5, n_select=2)
    tuned_best = [tune_model(i, optimize='RMSE',n_iter = 3) for i in best]
    blend_best = blend_models(estimator_list=tuned_best,optimize='RMSE')
    globals()['final_model_{}'.format(idx)] = finalize_model(blend_best)
    # f = finalize_model(best_model)
    
#     # attach final model to a dictionary
#     final_model[i] = f
    save_model(globals()['final_model_{}'.format(idx)],model_name='model'+str(idx),verbose=False)
#     # save transformation pipeline and model as pickle file 
#     save_model(f, model_name='trained_models/' + str(i), verbose=False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.5013,0.4166,0.6454,0.0609,0.0236,0.0191
1,0.4936,0.4065,0.6376,0.0637,0.0233,0.0188
2,0.4959,0.4013,0.6335,0.0619,0.0232,0.0189
3,0.4862,0.3910,0.6253,0.0663,0.0228,0.0185
4,0.4989,0.4112,0.6413,0.0566,0.0234,0.0189
5,0.4845,0.3889,0.6236,0.0589,0.0228,0.0184
6,0.4895,0.3916,0.6258,0.0658,0.0229,0.0186
7,0.4940,0.4083,0.6390,0.0692,0.0233,0.0188
8,0.4989,0.4095,0.6399,0.0646,0.0234,0.0190


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 

/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.54643e-13): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
 

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 

/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.55399e-13): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
 

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 

/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.49018e-13): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/server-003/anac

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 

/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/server-003/anaconda3/envs/pycaret/lib/python3.8/site-packages/sklearn/linear_model/_huber.py:296: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

In [10]:
for idx in range(1,15):
    globals()['final_model_{}'.format(idx)] = load_model('./model'+str(idx))

# model_1 = load_model('./model1') 


Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [36]:
pred_total = predict_model(final_model_1, data=test_df_drop)
pred_total = pred_total['Label']
for idx in range(2,10):
    pred = predict_model(globals()['final_model_{}'.format(idx)] , data = test_df_drop)
    pred = pred['Label']
    pred_total = pd.concat([pred_total,pred],axis=1)
    

#     pred = predict_model(final_model_1, data=test_df_drop)
    


In [38]:
for idx in range(10,15):
    pred = predict_model(globals()['final_model_{}'.format(idx)] , data = test_df_drop)
    pred = pred['Label']
    pred_total = pd.concat([pred_total,pred],axis=1)
pred_total

,Label,Label,Label,Label,Label,Label,Label,Label,Label,Label,Label,Label,Label,Label
0,1.372874,1.107695,1.073666,13.615711,31.346755,16.529176,3.098484,-26.087456,-26.131795,-22.068604,24.324816,-26.202408,-26.023977,-26.033691
1,1.417196,1.156158,1.141215,13.615710,31.107446,16.529176,3.120420,-26.323917,-26.168222,-22.420786,24.324816,-26.330432,-26.079994,-26.293103
2,1.381837,1.076164,1.051172,13.713265,31.502512,16.529177,3.038560,-26.050274,-26.114319,-22.468041,24.324818,-26.072472,-26.020180,-26.164200
3,1.389657,1.097023,1.018850,13.713266,32.029542,16.529177,3.176205,-25.997171,-25.990517,-21.804754,24.324820,-25.887416,-25.926986,-25.935463
4,1.340511,1.050660,0.916303,13.713265,31.547043,16.529177,3.123129,-26.002269,-26.015455,-22.238338,24.324819,-26.008182,-26.074048,-25.869195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,1.299019,1.017708,0.996471,13.615705,31.050004,16.529177,3.171362,-26.317458,-26.454803,-22.822807,24.324816,-26.239598,-26.315442,-26.352551
39604,1.293207,0.978756,0.952228,13.615706,31.148110,16.529177,3.138270,-26.287230,-26.341640,-22.967591,24.324816,-26.214083,-26.221260,-26.263684
39605,1.299382,0.998626,0.948438,13.615705,31.289576,16.529177,3.152596,-26.321351,-26.492513,-22.704830,24.324816,-26.395919,-26.296552,-26.347423
39606,1.294563,0.961096,0.909762,13.615706,31.054887,16.529176,3.169344,-26.414686,-26.313532,-22.667943,24.324816,-26.228825,-26.423278,-26.422601


In [31]:
# result = pred['Label']
# result_2 = pred['Label']
# result_3=pd.concat([result, result_2], axis=1)

In [39]:
submission_df = pd.read_csv(CFG.submission)
submission_df
submission_df.iloc[:, 1:] = pred_total
# submission_df.to_csv('./submit_rev3.csv', index=False)

In [40]:
submission_df

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.372874,1.107695,1.073666,13.615711,31.346755,16.529176,3.098484,-26.087456,-26.131795,-22.068604,24.324816,-26.202408,-26.023977,-26.033691
1,TEST_00002,1.417196,1.156158,1.141215,13.615710,31.107446,16.529176,3.120420,-26.323917,-26.168222,-22.420786,24.324816,-26.330432,-26.079994,-26.293103
2,TEST_00003,1.381837,1.076164,1.051172,13.713265,31.502512,16.529177,3.038560,-26.050274,-26.114319,-22.468041,24.324818,-26.072472,-26.020180,-26.164200
3,TEST_00004,1.389657,1.097023,1.018850,13.713266,32.029542,16.529177,3.176205,-25.997171,-25.990517,-21.804754,24.324820,-25.887416,-25.926986,-25.935463
4,TEST_00005,1.340511,1.050660,0.916303,13.713265,31.547043,16.529177,3.123129,-26.002269,-26.015455,-22.238338,24.324819,-26.008182,-26.074048,-25.869195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,TEST_39604,1.299019,1.017708,0.996471,13.615705,31.050004,16.529177,3.171362,-26.317458,-26.454803,-22.822807,24.324816,-26.239598,-26.315442,-26.352551
39604,TEST_39605,1.293207,0.978756,0.952228,13.615706,31.148110,16.529177,3.138270,-26.287230,-26.341640,-22.967591,24.324816,-26.214083,-26.221260,-26.263684
39605,TEST_39606,1.299382,0.998626,0.948438,13.615705,31.289576,16.529177,3.152596,-26.321351,-26.492513,-22.704830,24.324816,-26.395919,-26.296552,-26.347423
39606,TEST_39607,1.294563,0.961096,0.909762,13.615706,31.054887,16.529176,3.169344,-26.414686,-26.313532,-22.667943,24.324816,-26.228825,-26.423278,-26.422601


In [43]:
submission_df.to_csv('./submit.csv', index=False)